In [ ]:
import numpy as np
import pandas as pd
import os
import torch

os.chdir("../")

from src.conf import LAYERS_DIMS, MODEL_FEATURES
from src.utils import get_device, read_data_from_s3
from src.data import DataLoaders
from src.models import FraudAutoEncoder
from src.models import Model_Inference

In [ ]:
read_data_from_s3('data/typical_customers.gzip').drop(columns=["Ids", "Y"], axis=1).values

## Data Gen

In [ ]:
n_, k_ = 10_000, 300

_n = 1_000

In [ ]:
_data = np.concatenate(
    (
        np.concatenate(
            (np.random.uniform(**{"low": -1, "high": 0, "size": [_n, k_]}), np.ones((_n, 1))),
            axis=1
        ),
        np.concatenate(
            (np.random.uniform(**{"low": 0, "high": 1, "size": [n_, k_]}), np.zeros((n_, 1))),
            axis=1
        )
    )
)

all_customers_data = pd.DataFrame(
    data=_data,
    columns=[*[f"X_{str(i).zfill(3)}" for i in range(1, 1+k_)], "Y"]
).sample(frac=1).reset_index().rename(columns={"index": "Ids"})

all_customers_data.to_parquet("./data/all_customers.gzip", compression="gzip")

all_customers_data.query("Y == 0").to_parquet("./data/typical_customers.gzip", compression="gzip")


In [ ]:
all_customers_data

## Model Loading

In [ ]:
# Model_Inference("on_real_data")._error_eval(X.values[98])

In [ ]:
!nvidia-smi

In [ ]:
import s3fs
import yaml

with open('./data/s3_credentials.yaml', 'r') as file:
    access_keys = yaml.safe_load(file)

fs = s3fs.S3FileSystem(
    key = access_keys["key"],
    secret = access_keys["secret"]
)

pd.read_parquet(
    fs.open(fs.ls('customer-data-platform-retail/test_data/')[0])
)

In [ ]:
import awswrangler
import boto3

boto3_session = boto3.Session(
    aws_access_key_id=access_keys["key"],
    aws_secret_access_key=access_keys["secret"],
    region_name=access_keys["region"]
)

awswrangler.s3.upload(
    local_file="/home/onyxia/work/fraud_detection/models/best_model_on_real_data.ckpt",
    path="s3://models-factory/best_model_on_real_data.ckpt",
    boto3_session= boto3_session
)

In [ ]:
import boto3

s3 = boto3.client(
    's3',
    aws_access_key_id=access_keys["key"], 
    aws_secret_access_key=access_keys["secret"]
)

with open("/home/onyxia/work/fraud_detection/models/best_model_on_real_data.ckpt", "rb") as f:
    s3.upload_fileobj(f, "models-registry", "best_model_on_real_data.ckpt")

In [ ]:
fs.ls(f"customer-data-platform-retail/test_data/{'typical_customers.gzip'}")[0]